In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('final_df2.csv',index_col=0)

In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [4]:
from sklearn.preprocessing import StandardScaler

std_df = StandardScaler().fit_transform(X)
std_df = pd.DataFrame(std_df, index=X.index, columns=X.columns)
df2 = std_df
df2.head()

,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,RXFP3,...,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC
0,0.886827,0.349044,0.159261,-1.586876,-0.783517,-0.651792,-0.192552,-0.688943,1.790315,0.606942,...,0.618291,-0.244858,0.087421,-0.441970,0.805609,0.084007,-0.853409,-0.767943,-0.097567,0.208945
1,-0.076357,0.902533,0.026440,1.624105,1.396133,0.195322,0.394281,0.733565,1.460883,-0.288978,...,0.083736,-0.957388,0.095699,0.972672,0.691112,1.004964,-0.178898,-0.945141,0.843130,-1.062326
2,1.160544,0.875732,4.027129,0.072768,0.488563,-0.707819,-1.258655,2.353938,1.514641,-0.194159,...,0.561794,-0.680456,-0.348212,0.766775,-0.167253,0.878351,1.027238,-0.487267,0.630930,-0.273803
3,-0.093755,0.023215,-1.310286,0.819120,-0.815949,-0.407544,0.793497,-0.158602,-0.161729,1.060372,...,2.103530,-1.246807,-1.676863,0.213831,-1.585708,0.933448,0.740015,-1.050423,-0.987469,-0.644158
4,-0.249586,-0.633865,-0.325074,0.155425,0.019339,-0.462644,-0.576603,-1.569976,-1.943742,0.022223,...,-1.478282,0.569776,-0.794047,-0.079013,0.559841,-0.793164,-1.567624,0.733505,-0.957059,0.428773


In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_over, y_train_over = smote.fit_resample(X, y)
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", X.shape, y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 값의 분포 :\n',pd.Series(y_train_over).value_counts() )

SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (590, 17814) (590,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : (1058, 17814) (1058,)
SMOTE 적용 후 값의 분포 :
 1    529
0    529
Name: label, dtype: int64


In [6]:
print('SMOTE 적용 전 값의 분포 :\n',pd.Series(y).value_counts() )

SMOTE 적용 전 값의 분포 :
 1    529
0     61
Name: label, dtype: int64


In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=516)
pca_array = pca.fit_transform(X_train_over)


In [8]:
pca = PCA()
pca.fit(X_train_over)
cumsum = np.cumsum(pca.explained_variance_ratio_)
# 주성분의 개수를 지정하지 않고 찾아줌
# 분산의 비율을 누적해서 더한 값이 95% 이상인 주성분의 개수
# 즉, 분산의 95%이상 유지
d = np.argmax(cumsum >= 0.99) + 1

In [9]:
d

516

In [10]:
dec=PCA(0.99)
x2=dec.fit_transform(X_train_over)
x2.shape

(1058, 516)

In [11]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test=train_test_split(pca_array,y_train_over,test_size= 0.2, random_state=42)

In [12]:
def estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC()],tr_slicer=(None,None),tst_slicer=(None,None),loops=300,scorer=accuracy_score,x=x,y=y):
    training_score={}
    testing_score={}
    timing={}
    for cl in estimators:
        cl_name=cl.__class__.__name__
        training_score[cl_name]=[]
        testing_score[cl_name]=[]
        timing[cl_name]=[]
    for i in range(loops):
        k1=time()
        x_train, x_test, y_train, y_test=train_test_split(x,y,random_state=i)
        for cl in estimators:
            a=time()
            cl_name=cl.__class__.__name__
            clean_cl=clone(cl)
            clean_cl.fit(x_train[tr_slicer[0]:tr_slicer[1]],y_train[tr_slicer[0]:tr_slicer[1]])
            training_score[cl_name].append(scorer(y_train[tr_slicer[0]:tr_slicer[1]],clean_cl.predict(x_train[tr_slicer[0]:tr_slicer[1]])))
            testing_score[cl_name].append(scorer(y_test[tst_slicer[0]:tst_slicer[1]],clean_cl.predict(x_test[tst_slicer[0]:tst_slicer[1]])))
            b=time()                                
            timing[cl_name].append(b-a)
        k2=time()
        print(f'number {i} out of {loops} took {k2-k1} seconds')
    global training_score_df
    training_score_df=pd.DataFrame(training_score)
    global testing_score_df
    testing_score_df=pd.DataFrame(testing_score)
    global timing_df
    timing_df=pd.DataFrame(timing)

In [13]:
estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC(),LogisticRegression()],x=x,y=y)

number 0 out of 300 took 2.4903407096862793 seconds
number 1 out of 300 took 2.4314992427825928 seconds
number 2 out of 300 took 2.4783735275268555 seconds
number 3 out of 300 took 2.453441619873047 seconds
number 4 out of 300 took 2.4644103050231934 seconds
number 5 out of 300 took 2.6628806591033936 seconds
number 6 out of 300 took 2.5212595462799072 seconds
number 7 out of 300 took 2.627974271774292 seconds
number 8 out of 300 took 2.454437732696533 seconds
number 9 out of 300 took 2.476379632949829 seconds
number 10 out of 300 took 2.57910418510437 seconds
number 11 out of 300 took 2.509291410446167 seconds
number 12 out of 300 took 2.4195311069488525 seconds
number 13 out of 300 took 2.502310276031494 seconds
number 14 out of 300 took 2.594064950942993 seconds
number 15 out of 300 took 2.5013134479522705 seconds
number 16 out of 300 took 2.5132806301116943 seconds
number 17 out of 300 took 2.701777935028076 seconds
number 18 out of 300 took 2.625978469848633 seconds
number 19 out 

number 156 out of 300 took 2.3995845317840576 seconds
number 157 out of 300 took 2.4015793800354004 seconds
number 158 out of 300 took 2.383626937866211 seconds
number 159 out of 300 took 2.3955960273742676 seconds
number 160 out of 300 took 2.3826303482055664 seconds
number 161 out of 300 took 2.4205286502838135 seconds
number 162 out of 300 took 2.38262939453125 seconds
number 163 out of 300 took 2.3995847702026367 seconds
number 164 out of 300 took 2.416539192199707 seconds
number 165 out of 300 took 2.4025776386260986 seconds
number 166 out of 300 took 2.445462703704834 seconds
number 167 out of 300 took 2.4813649654388428 seconds
number 168 out of 300 took 2.4125494956970215 seconds
number 169 out of 300 took 2.397590160369873 seconds
number 170 out of 300 took 2.384625196456909 seconds
number 171 out of 300 took 2.4005825519561768 seconds
number 172 out of 300 took 2.3926031589508057 seconds
number 173 out of 300 took 2.400583028793335 seconds
number 174 out of 300 took 2.4285066

In [14]:
training_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.0,300.0,300.000000,300.0
mean,1.0,1.0,0.999022,1.0
std,0.0,0.0,0.000767,0.0
min,1.0,1.0,0.998423,1.0
25%,1.0,1.0,0.998423,1.0
50%,1.0,1.0,0.998423,1.0
75%,1.0,1.0,1.000000,1.0
max,1.0,1.0,1.000000,1.0


In [15]:
testing_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.000000,300.000000
mean,0.992233,0.996509,0.997720,0.998113
std,0.006893,0.004117,0.002740,0.002700
min,0.962264,0.981132,0.990566,0.990566
25%,0.990566,0.995283,0.995283,0.995283
50%,0.995283,0.995283,1.000000,1.000000
75%,0.995283,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [16]:
timing_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.000000,300.000000
mean,0.541628,1.793851,0.053746,0.021782
std,0.014169,0.029495,0.002660,0.002718
min,0.513627,1.769270,0.048868,0.017951
25%,0.534321,1.780241,0.051862,0.020694
50%,0.540056,1.786225,0.053855,0.020945
75%,0.546538,1.795201,0.054853,0.021942
max,0.644278,2.026583,0.072805,0.040892


In [17]:
def estimators_repeater2(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC()],tr_slicer=(None,None),tst_slicer=(None,None),loops=300,scorer=recall_score,x=x,y=y):
    training_score={}
    testing_score={}
    timing={}
    for cl in estimators:
        cl_name=cl.__class__.__name__
        training_score[cl_name]=[]
        testing_score[cl_name]=[]
        timing[cl_name]=[]
    for i in range(loops):
        k1=time()
        x_train, x_test, y_train, y_test=train_test_split(x,y,random_state=i)
        for cl in estimators:
            a=time()
            cl_name=cl.__class__.__name__
            clean_cl=clone(cl)
            clean_cl.fit(x_train[tr_slicer[0]:tr_slicer[1]],y_train[tr_slicer[0]:tr_slicer[1]])
            training_score[cl_name].append(scorer(y_train[tr_slicer[0]:tr_slicer[1]],clean_cl.predict(x_train[tr_slicer[0]:tr_slicer[1]])))
            testing_score[cl_name].append(scorer(y_test[tst_slicer[0]:tst_slicer[1]],clean_cl.predict(x_test[tst_slicer[0]:tst_slicer[1]])))
            b=time()                                
            timing[cl_name].append(b-a)
        k2=time()
        print(f'number {i} out of {loops} took {k2-k1} seconds')
    global training_score_df
    training_score_df=pd.DataFrame(training_score)
    global testing_score_df
    testing_score_df=pd.DataFrame(testing_score)
    global timing_df
    timing_df=pd.DataFrame(timing)

In [18]:
estimators_repeater2(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC(),LogisticRegression()],x=x,y=y)

number 0 out of 300 took 2.4095571041107178 seconds
number 1 out of 300 took 2.404571294784546 seconds
number 2 out of 300 took 2.419532060623169 seconds
number 3 out of 300 took 2.4135470390319824 seconds
number 4 out of 300 took 2.412551164627075 seconds
number 5 out of 300 took 2.3926031589508057 seconds
number 6 out of 300 took 2.442469835281372 seconds
number 7 out of 300 took 2.449450731277466 seconds
number 8 out of 300 took 2.391605854034424 seconds
number 9 out of 300 took 2.4135475158691406 seconds
number 10 out of 300 took 2.409557580947876 seconds
number 11 out of 300 took 2.424518585205078 seconds
number 12 out of 300 took 2.4065659046173096 seconds
number 13 out of 300 took 2.405568838119507 seconds
number 14 out of 300 took 2.42850661277771 seconds
number 15 out of 300 took 2.404572010040283 seconds
number 16 out of 300 took 2.409558057785034 seconds
number 17 out of 300 took 2.405569553375244 seconds
number 18 out of 300 took 2.420527935028076 seconds
number 19 out of 3

number 156 out of 300 took 2.4205286502838135 seconds
number 157 out of 300 took 2.3926026821136475 seconds
number 158 out of 300 took 2.417537212371826 seconds
number 159 out of 300 took 2.3816323280334473 seconds
number 160 out of 300 took 2.409557819366455 seconds
number 161 out of 300 took 2.4135472774505615 seconds
number 162 out of 300 took 2.405568838119507 seconds
number 163 out of 300 took 2.4035754203796387 seconds
number 164 out of 300 took 2.404571056365967 seconds
number 165 out of 300 took 2.4275104999542236 seconds
number 166 out of 300 took 2.419529914855957 seconds
number 167 out of 300 took 2.4005825519561768 seconds
number 168 out of 300 took 2.4105546474456787 seconds
number 169 out of 300 took 2.3995847702026367 seconds
number 170 out of 300 took 2.4215259552001953 seconds
number 171 out of 300 took 2.419532299041748 seconds
number 172 out of 300 took 2.432495355606079 seconds
number 173 out of 300 took 2.3975894451141357 seconds
number 174 out of 300 took 2.420527

In [19]:
testing_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.0,300.000000,300.000000,300.000000
mean,1.0,0.995191,0.995331,0.996112
std,0.0,0.006282,0.005646,0.005560
min,1.0,0.958333,0.979167,0.979167
25%,1.0,0.990361,0.990385,0.990476
50%,1.0,1.000000,1.000000,1.000000
75%,1.0,1.000000,1.000000,1.000000
max,1.0,1.000000,1.000000,1.000000
